In [1]:
! pip install pydub

In [2]:
import numpy as np
import pandas as pd
from pydub import AudioSegment
from pydub.generators import Sine
import torch
import colorsys

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [4]:
df = pd.read_csv('/content/galaxy_image_pixel_data.csv')  # Replace with your CSV file path

# Take the first 1000 rows
df_subset = df.head(1000)

In [5]:
# Create an empty audio segment
audio = AudioSegment.silent(duration=0)

### Function to map RGB values to frequencies

In [6]:
def map_frequency(value, min_freq, max_freq, offset):
    return offset + (value / 255.0) * (max_freq - min_freq)

## Generate an AudioSegment from RGB values

In [7]:
def generate_audio_from_rgb(rgb_data, duration_ms=100):
    audio = AudioSegment.silent(duration=0)  # Initialize an empty audio segment

    # Defining frequency ranges
    rgb_frequency_range = {
        'r': {'min': 30, 'max': 500, 'offset': 60},
        'g': {'min': 500, 'max': 2000, 'offset': 250},
        'b': {'min': 2000, 'max': 10000, 'offset': 1000},
    }

    # Generate sine waves for each RGB value
    for _, row in rgb_data.iterrows():
        freq_r = map_frequency(row['R'], rgb_frequency_range['r']['min'], rgb_frequency_range['r']['max'], rgb_frequency_range['r']['offset'])
        freq_g = map_frequency(row['G'], rgb_frequency_range['g']['min'], rgb_frequency_range['g']['max'], rgb_frequency_range['g']['offset'])
        freq_b = map_frequency(row['B'], rgb_frequency_range['b']['min'], rgb_frequency_range['b']['max'], rgb_frequency_range['b']['offset'])

        sine_r = Sine(freq_r).to_audio_segment(duration=duration_ms, volume=row['R'] / 255 * 20 - 20)
        sine_g = Sine(freq_g).to_audio_segment(duration=duration_ms, volume=row['G'] / 255 * 20 - 20)
        sine_b = Sine(freq_b).to_audio_segment(duration=duration_ms, volume=row['B'] / 255 * 20 - 20)

        # Combine the sine waves for R, G, and B into a single segment
        combined = sine_r.overlay(sine_g).overlay(sine_b)
        audio += combined  # Append to the audio segment

    return audio

## Export the final sound file

In [11]:
output_path = "output_sound_1000_rows_frequency_mapping.wav"
audio = generate_audio_from_rgb(df_subset, duration_ms=100)
audio.export(output_path, format="wav")
print("WAV file saved at:", output_path)

WAV file saved at: output_sound_1000_rows_frequency_mapping.wav
